# Set Options

In [ ]:
### Set options here
#Structural options
market = 'FullYear' #Choose from ['Balancing', 'DayAhead', 'FullYear', 'Investment']
SCENARIO = 'all' #Add scenario to read file name
YEAR = 'full' #Add year to read file name (e.g. '2025', '2035', 'full')
SUBSET = 'full'
year = 2045 #Year to be displayed
LINES = 'CongestionFlow' #Choose from: ['Capacity', 'Flow', 'CongestionFlow']. For 'CongestionFlow', exo_end automatically switches to 'Total'.
exo_end = 'Total' # Choose from ['Endogenous', 'Exogenous', 'Total']. For 'CongestionFlow', exo_end automatically switches to 'Total'.
S = 'S12' #Season 
T = 'T103' #Hour


#Visual options
label_min = 0 #Minimum transmission capacity (GW) shown on map in text
font_line = 10 #Font size of transmission line labels
font_hub = 8 #Font size of hub labels
font_region = 12 #Font size of region labels
line_decimals = 1 #Number of decimals shown for line capacities
hub_decimals = 0 #Number of decimals shown for hub capacities

#Colours
background_colour = 'white'
regions_ext_colour = 'lightgrey'
regions_model_colour = 'grey'
region_text = 'black'
capline_colour = 'orange'
flowline_colour = ['#74c476','#feb24c','#a50f15'] #NB: Change legend colours accordingly, in the cell under "Add Legend": lines 43-45
flowline_breaks = [0, 50, 94.999, 100]
line_text = 'black'
hub_display = True
hub_colour = 'lightblue'
hub_text = 'black'

# Import Packages

In [ ]:
from pathlib import Path
import sys
import os
import pandas as pd
import numpy as np
import folium
from folium import plugins
from IPython.display import HTML, display
import json
from folium.features import DivIcon #For text labels on hubs
from IPython.display import display, HTML
from csv import reader
display(HTML(data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }
</style>
"""))

# Import Files

In [ ]:
#Set file names
project_dir = Path('./input')
map_name = 'Transmission_' + LINES + '_' + str(year) + '_Map.html'
flow_file = 'FlowElectricityHourly_'+ SCENARIO + '_' + YEAR + '_' + SUBSET + '.csv'
transcap_file = 'CapacityPowerTransmission_'+ SCENARIO + '_' + YEAR + '_'+ SUBSET + '.csv'
generation_file = 'CapacityGeneration_'+ SCENARIO + '_' + YEAR + '_' + SUBSET + '.csv'

#Load regions files 
df_unique = pd.read_csv(project_dir/'transmission_files/coordinates_RRR.csv')
df_region = df_unique.loc[df_unique['Type'] == 'region', ]
regions_model_layer = f'{project_dir}/transmission_files/regions.geojson' #Polygons of modelled countries
regions_ext_layer = f'{project_dir}/transmission_files/countries_outside_model.geojson' #Polygons of neighbouring countries

#Transmission capacity data
df_capacity = pd.read_csv(str(project_dir) + '/results/' + str(market) + '/' + str(transcap_file), sep = ',', quotechar = '"') 
#Transmission flow data
df_flow = pd.read_csv(str(project_dir) + '/results/' + str(market) + '/' + str(flow_file), sep = ',', quotechar = '"')
#Generation capacity data
df_capgen = pd.read_csv(str(project_dir) + '/results/' + str(market) + '/' + str(generation_file), sep = ',', quotechar = '"') 

# Replace "EPS" with 0

In [ ]:
#Replace possible "Eps" with 0
df_capacity.Val=df_capacity.Val.replace('Eps', 0)
df_capacity.Val=pd.to_numeric(df_capacity.Val)
df_flow.Val=df_flow.Val.replace('Eps', 0)
df_flow.Val=pd.to_numeric(df_flow.Val)
df_capgen.Val=df_capgen.Val.replace('Eps', 0)
df_capgen.Val=pd.to_numeric(df_capgen.Val)

# Add Coordinates + Select Time + Convert Units

In [ ]:
#Flows
if LINES == 'Flow' or LINES == 'CongestionFlow': #Skip this cell in case LINES == 'Capacity'
    #Keep only data from moment of interest
    df_flow = df_flow.loc[df_flow['Y'] == year] 
    df_flow = df_flow.loc[df_flow['SSS'] == S,]
    df_flow = df_flow.loc[df_flow['TTT'] == T, ]
    for i,row in df_flow.iterrows():
        for j in range(0,len(df_unique)):
            if df_flow.loc[i,'IRRRE'] == df_unique.loc[j, 'RRR']:
                df_flow.loc[i,'LatExp'] = df_unique.loc[j, 'Lat']
                df_flow.loc[i,'LonExp'] = df_unique.loc[j, 'Lon']
            if df_flow.loc[i,'IRRRI'] == df_unique.loc[j, 'RRR']:
                df_flow.loc[i,'LatImp'] = df_unique.loc[j, 'Lat']
                df_flow.loc[i,'LonImp'] = df_unique.loc[j, 'Lon']

    #Convert flow from MWh to GWh
    df_flow['Val'] = df_flow['Val'] / 1000
    df_flow = df_flow.reset_index(drop = True)
    if len(df_flow) == 0:
        print("Error: Timestep not in data; check year, S and T.")
        sys.exit()

In [ ]:
#Generation Capacities
df_capgen = df_capgen.merge(df_unique, on = 'RRR', how = 'left', left_index = True).reset_index(drop = True) #Add coordinates of each region
df_capgen = df_capgen.loc[df_capgen['Y'] == year] #Keep only data from year of interest
df_hubgen = df_capgen.loc[df_capgen['Type']== 'hub',] #Keep only hub data 
df_hubgen_agg = pd.DataFrame(df_hubgen.groupby(['Y', 'C', 'RRR', 'Lat', 'Lon'])['Val'].sum().reset_index()) #Sum all capacities (of different wind turbines) at each location

In [ ]:
#Transmission Capacities
if LINES == 'Capacity' or LINES == 'CongestionFlow': #Skip this cell in case LINES == 'Flow'
    df_capacity = df_capacity.loc[df_capacity['Y'] == year, ].reset_index(drop = True) #Keep only data from year of interest

    if exo_end == 'Total' or LINES == 'CongestionFlow':
        col_keep = list(np.delete(np.array(df_capacity.columns),np.where((df_capacity.columns == 'VARIABLE_CATEGORY') | \
                                    (df_capacity.columns == 'Val')) )) #Create list with all columns except 'Variable_Category' and 'Val'
        df_capacity = pd.DataFrame(df_capacity.groupby(col_keep)['Val'].sum().reset_index() )#Sum exogenous and endogenous capacity for each region
    if exo_end == 'Endogenous' and LINES != 'CongestionFlow':
        df_capacity = df_capacity.loc[df_capacity['VARIABLE_CATEGORY'] == 'ENDOGENOUS', ]
    if exo_end == 'Exogenous' and LINES != 'CongestionFlow':
        df_capacity = df_capacity.loc[df_capacity['VARIABLE_CATEGORY'] == 'EXOGENOUS', ]

    for i,row in df_capacity.iterrows():
        for j in range(0,len(df_unique)):
            if df_capacity.loc[i,'IRRRE'] == df_unique.loc[j, 'RRR']:
                df_capacity.loc[i,'LatExp'] = df_unique.loc[j, 'Lat']
                df_capacity.loc[i,'LonExp'] = df_unique.loc[j, 'Lon']
            if df_capacity.loc[i,'IRRRI'] == df_unique.loc[j, 'RRR']:
                df_capacity.loc[i,'LatImp'] = df_unique.loc[j, 'Lat']
                df_capacity.loc[i,'LonImp'] = df_unique.loc[j, 'Lon']
    if len(df_capacity) == 0:
        print("Error: No capacity found. Check year and exo_end.")
        sys.exit()

# Calculate Congestion

In [ ]:
if LINES == 'CongestionFlow': #Skip this cell in case LINES != 'CongestionFlow'
    df_flow = pd.merge(df_flow, df_capacity[['Y', 'C', 'IRRRE', 'IRRRI', 'Val']], on = ['Y', 'C', 'IRRRE', 'IRRRI'], how = 'left')
    df_flow.rename(columns={'Val_x': 'Val', 'Val_y' : 'Capacity'}, inplace = True)
    df_flow['Congestion'] = df_flow['Val'] / df_flow['Capacity'] * 100

    #Create colour codes for congestion of lines
    df_flow['Colour'] = pd.cut(df_flow['Congestion'], bins = flowline_breaks, labels = flowline_colour )

# Create Map Features

In [ ]:
#When capacity is not the same in both directions, display one:
for i,row in df_capacity.iterrows():
    for k,row in df_capacity.iterrows():
        if (df_capacity.loc[k,'IRRRE'] == df_capacity.loc[i,'IRRRI']) & (df_capacity.loc[k,'IRRRI'] == df_capacity.loc[i,'IRRRE']) & (df_capacity.loc[k,'Val'] != df_capacity.loc[i,'Val']):
            df_capacity.loc[i,'Val'] = df_capacity.loc[k,'Val']

In [ ]:
#Create column that is used as hub display size on the map
if len(df_hubgen_agg) == 0:
    df_hubgen_agg['Map_size'] = 0
else: df_hubgen_agg['Map_size'] = df_hubgen_agg['Val'] / min(df_hubgen_agg['Val']) + 5

In [ ]:
#Define centre of each transmission line
if LINES == 'Flow' or LINES == 'CongestionFlow': #Skip this cell in case LINES == 'Capacity'
    df_flow['LatMid'] = (df_flow['LatImp'] + df_flow['LatExp']) /2
    df_flow['LonMid'] = (df_flow['LonImp'] + df_flow['LonExp']) /2
if LINES == 'Capacity' or LINES == 'CongestionFlow': #Skip this cell in case LINES == 'Flow'
    df_capacity['LatMid'] = (df_capacity['LatImp'] + df_capacity['LatExp']) /2
    df_capacity['LonMid'] = (df_capacity['LonImp'] + df_capacity['LonExp']) /2

In [ ]:
#Create map 
map_center = [55.220228, 10.419778]
m = folium.Map(location= map_center, zoom_start=5, tiles='')
#Add background layers (sea, regions in model, countries outside of model)
folium.Polygon(locations = [[-90,-180], [90,-180], [90,180], [-90,180]], color = background_colour, fill_color = background_colour, opacity = 1, fill_opacity = 1 ).add_to(m) #Background
folium.GeoJson(data = regions_model_layer, name = 'regions', \
               style_function = lambda x:{'fillColor': regions_model_colour, 'fillOpacity': 0.5, 'color': regions_model_colour, 'weight':1}).add_to(m) #Regions within model
folium.GeoJson(data = regions_ext_layer, name = 'x', \
               style_function = lambda x:{'fillColor': regions_ext_colour, 'fillOpacity': 0.5, 'color': regions_ext_colour, 'weight':1}).add_to(m) #Neighbouring countries

In [ ]:
#Add capacity lines
if LINES == 'Capacity':
    for i,row in df_capacity.iterrows():
        folium.PolyLine(([df_capacity.loc[i,'LatExp'], df_capacity.loc[i,'LonExp']], \
                             [df_capacity.loc[i,'LatImp'],df_capacity.loc[i,'LonImp']]), \
                            color=capline_colour, line_cap = 'butt', weight=df_capacity.loc[i,'Val']/5, opacity=1).add_to(m)  
        if df_capacity.loc[i,'Val'] > label_min:
            if line_decimals == 0:
                folium.Marker(location=[df_capacity.loc[i,'LatMid'], df_capacity.loc[i,'LonMid']],
                          icon=DivIcon(
                              icon_size=(150,36), 
                                       icon_anchor=(11,7),
                     html='<div style="font-size: {}pt; color : {}">{}</div>'.format(font_line, line_text, df_capacity.loc[i,'Val'].round(line_decimals).astype(int)))).add_to(m)
            else: 
                folium.Marker(location=[df_capacity.loc[i,'LatMid'], df_capacity.loc[i,'LonMid']],
                          icon=DivIcon(
                              icon_size=(150,36), 
                                       icon_anchor=(11,7),
                     html='<div style="font-size: {}pt; color : {}">{}</div>'.format(font_line, line_text, round(df_capacity.loc[i,'Val'],line_decimals)))).add_to(m)
#Add flows (single colour)                
if LINES == 'Flow':
    attr = {'font-weight': 'bold', 'font-size': '24'}
    for i,row in df_flow.iterrows():
        flow = folium.PolyLine(([df_flow.loc[i,'LatExp'], df_flow.loc[i,'LonExp']], \
                             [df_flow.loc[i,'LatImp'],df_flow.loc[i,'LonImp']]), \
                            color=capline_colour, line_cap = 'butt', weight=df_flow.loc[i,'Val']/5, opacity=1).add_to(m)   
        plugins.PolyLineTextPath(flow, '\u2192', repeat=False ,center = True, offset=6, orientation = -90, attributes=attr).add_to(m)  #Arrow
        if df_flow.loc[i,'Val'] > label_min:
            if line_decimals == 0:
                folium.Marker(location=[df_flow.loc[i,'LatMid'], df_flow.loc[i,'LonMid']],
                          icon=DivIcon(
                              icon_size=(150,36), 
                                       icon_anchor=(11,7),
                     html='<div style="font-size: {}pt; color : {}">{}</div>'.format(font_line, line_text, df_flow.loc[i,'Val'].round(line_decimals).astype(int)))).add_to(m)
            else: 
                folium.Marker(location=[df_flow.loc[i,'LatMid'], df_flow.loc[i,'LonMid']],
                          icon=DivIcon(
                              icon_size=(150,36), 
                                       icon_anchor=(11,7),
                     html='<div style="font-size: {}pt; color : {}">{}</div>'.format(font_line, line_text, round(df_flow.loc[i,'Val'],line_decimals)))).add_to(m)    
#Add flows (colour based on congestion)
if LINES == 'CongestionFlow':
    attr = {'font-weight': 'bold', 'font-size': '24'}
    for i,row in df_flow.iterrows():
        flow = folium.PolyLine(([df_flow.loc[i,'LatExp'], df_flow.loc[i,'LonExp']], \
                             [df_flow.loc[i,'LatImp'],df_flow.loc[i,'LonImp']]), \
                            color=df_flow.loc[i,'Colour'], line_cap = 'butt', weight=df_flow.loc[i,'Val']/5, opacity=1).add_to(m)   
        plugins.PolyLineTextPath(flow, '\u2192', repeat=False ,center = True, offset=6, orientation = -90, attributes=attr).add_to(m)  #Arrow
        if df_flow.loc[i,'Val'] > label_min:
            if line_decimals == 0:
                folium.Marker(location=[df_flow.loc[i,'LatMid'], df_flow.loc[i,'LonMid']],
                          icon=DivIcon(
                              icon_size=(150,36), 
                                       icon_anchor=(11,7),
                     html='<div style="font-size: {}pt; color : {}">{}</div>'.format(font_line, line_text, df_flow.loc[i,'Val'].round(line_decimals).astype(int)))).add_to(m)
            else: 
                folium.Marker(location=[df_flow.loc[i,'LatMid'], df_flow.loc[i,'LonMid']],
                          icon=DivIcon(
                              icon_size=(150,36), 
                                       icon_anchor=(11,7),
                     html='<div style="font-size: {}pt; color : {}">{}</div>'.format(font_line, line_text, round(df_flow.loc[i,'Val'],line_decimals)))).add_to(m)    

In [ ]:
#Add region names
for i,row in df_region.iterrows():
    folium.Marker(location=[df_region.loc[i,'Lat'], df_region.loc[i,'Lon']],
                  icon=DivIcon(
                      icon_size=(150,36), 
                               icon_anchor=(7,7),
     html='<div style="font-size: {}pt; color : {}">{}</div>'.format(font_region, region_text, df_region.loc[i,'RRR']))).add_to(m)  

In [ ]:
if hub_display == True:
    #Add hub capacities as bubbles
    for i,row in df_hubgen_agg.iterrows():
        folium.CircleMarker(
          location=[df_hubgen_agg.loc[i,'Lat'], df_hubgen_agg.loc[i,'Lon']],
          popup=df_hubgen_agg.loc[i,'RRR'],
          radius = df_hubgen_agg.loc[i, 'Map_size'],
          color= hub_colour,
          opacity = 0,
          fill=True,
          fill_color= hub_colour,
          fill_opacity = 1
       ).add_to(m)
        if hub_decimals == 0:
            folium.Marker(location=[df_hubgen_agg.loc[i,'Lat'], df_hubgen_agg.loc[i,'Lon']],
                          icon=DivIcon(
                              icon_size=(150,36), 
                                       icon_anchor=(7,7),
             html='<div style="font-size: {}pt; color : {}">{}</div>'.format(font_hub, hub_text, df_hubgen_agg.loc[i,'Val'].round(hub_decimals).astype(int)))).add_to(m)
        else:
            folium.Marker(location=[df_hubgen_agg.loc[i,'Lat'], df_hubgen_agg.loc[i,'Lon']],
                          icon=DivIcon(
                              icon_size=(150,36), 
                                       icon_anchor=(7,7),
             html='<div style="font-size: {}pt; color : {}">{}</div>'.format(font_hub, hub_text, round(df_hubgen_agg.loc[i,'Val'], hub_decimals)))).add_to(m)      

# Add Legend

In [ ]:
if LINES == 'CongestionFlow':
    from branca.element import Template, MacroElement

    template = """
    {% macro html(this, kwargs) %}

    <!doctype html>
    <html lang="en">
    <head>
      <meta charset="utf-8">
      <meta name="viewport" content="width=device-width, initial-scale=1">
      <title>jQuery UI Draggable - Default functionality</title>
      <link rel="stylesheet" href="//code.jquery.com/ui/1.12.1/themes/base/jquery-ui.css">

      <script src="https://code.jquery.com/jquery-1.12.4.js"></script>
      <script src="https://code.jquery.com/ui/1.12.1/jquery-ui.js"></script>

      <script>
      $( function() {
        $( "#maplegend" ).draggable({
                        start: function (event, ui) {
                            $(this).css({
                                right: "auto",
                                top: "auto",
                                bottom: "auto"
                            });
                        }
                    });
    });

      </script>
    </head>
    <body>


    <div id='maplegend' class='maplegend' 
        style='position: absolute; z-index:9999; border:2px solid grey; background-color:rgba(255, 255, 255, 1);
         border-radius:6px; padding: 10px; font-size:14px; right: 20px; bottom: 20px;'>

    <div class='legend-title'>Congestion rate</div>
    <div class='legend-scale'>
      <ul class='legend-labels'>
        <li><span style='background:#a50f15;opacity:0.7;'></span>Fully congested</li>
        <li><span style='background:#feb24c;opacity:0.7;'></span> 50-95% congested </li>
        <li><span style='background:#74c476;opacity:0.7;'></span> <50 % congested </li>

      </ul>
    </div>
    </div>

    </body>
    </html>

    <style type='text/css'>
      .maplegend .legend-title {
        text-align: left;
        margin-bottom: 5px;
        font-weight: bold;
        font-size: 90%;
        }
      .maplegend .legend-scale ul {
        margin: 0;
        margin-bottom: 5px;
        padding: 0;
        float: left;
        list-style: none;
        }
      .maplegend .legend-scale ul li {
        font-size: 80%;
        list-style: none;
        margin-left: 0;
        line-height: 18px;
        margin-bottom: 2px;
        }
      .maplegend ul.legend-labels li span {
        display: block;
        float: left;
        height: 16px;
        width: 30px;
        margin-right: 5px;
        margin-left: 0;
        border: 1px solid #999;
        }
      .maplegend .legend-source {
        font-size: 80%;
        color: #777;
        clear: both;
        }
      .maplegend a {
        color: #777;
        }
    </style>
    {% endmacro %}"""

    macro = MacroElement()
    macro._template = Template(template)

    m.get_root().add_child(macro)

# Save Output

In [ ]:
# Make Transmission_Map output folder
if not os.path.isdir('output/Transmission_Map/' + LINES + '/' + SCENARIO + '/' + market):
    os.makedirs('output/Transmission_Map/' + LINES + '/' + SCENARIO + '/' + market)

In [ ]:
output_dir = 'output/Transmission_Map/' + LINES + '/' + SCENARIO + '/' + market
m.save(output_dir + '/' +  map_name)

# Display Map

In [ ]:
m